<a href="https://colab.research.google.com/github/IzaakGagnon/Integrated_Information_Testing/blob/main/Custom_Partitioning_Schemes_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and Installs

In [1]:
!python -m pip install -U git+https://github.com/wmayner/pyphi.git@feature/iit-4.0
import numpy as np
import pyphi
import random
import time
pyphi.config.PROGRESS_BARS = False
pyphi.config.PARALLEL = False
pyphi.config.SHORTCIRCUIT_SIA = False
pyphi.config.WELCOME_OFF = True
pyphi.config.REPERTOIRE_DISTANCE = "GENERALIZED_INTRINSIC_DIFFERENCE"

  Cloning https://github.com/wmayner/pyphi.git (to revision feature/iit-4.0) to /tmp/pip-req-build-6__myjwi
  Running command git clone --filter=blob:none --quiet https://github.com/wmayner/pyphi.git /tmp/pip-req-build-6__myjwi
  Resolved https://github.com/wmayner/pyphi.git to commit 6b83cbdbbcdca75289415fe096adbac5f2ec7a4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general d

In [2]:
import functools
import itertools
from itertools import chain, product
import numpy as np
from more_itertools import distinct_permutations
from toolz import unique
from pyphi import combinatorics
from pyphi.cache import cache
from pyphi.conf import config, fallback
from pyphi.direction import Direction
from pyphi.models.cuts import (
    Bipartition,
    CompleteGeneralKCut,
    CompleteGeneralSetPartition,
    Cut,
    GeneralKCut,
    GeneralSetPartition,
    KPartition,
    Part,
    SystemPartition,
    Tripartition,
)
from pyphi.partition import system_partition_types
from pyphi.registry import Registry
from itertools import combinations

The generator of all unique k-partitions

In [28]:
def gen_seqs(partition):
    n = len(partition)
    elements = [0,1,2]
    all_sequences = itertools.product(elements, repeat=n)
    valid_sequences = [seq for seq in all_sequences if 0 in seq and 1 in seq]
    all_twos_sequence = (2,) * n
    valid_sequences.append(all_twos_sequence)
    return valid_sequences
@system_partition_types.register("ALL_K-PARTITIONS")
def generate_all_partitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
    for s in gen_seqs(partition):
      cm = seq_2_cm(s, partition,_node_indices)
      yield GeneralSetPartition(
                node_indices,
                cm,
                node_labels=node_labels,
                set_partition=partition,
            )

The generator of all unique bi-partitions:

In [29]:
def seq_2_cm(seq, partition,_node_indices):
  cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
  c = 0
  for part in partition:
    nonpart = list(_node_indices - set(part))
    if seq[c] == 0:                 ### If we are cutting the part's inputs
      source, target = nonpart, part
    else:                           ### If we are cutting the part's outputs
      source, target = part, nonpart
    cut_matrix[np.ix_(source, target)] = 1
    if seq[c] == 2:                 ### If we are cutting both inputs and outputs
      cut_matrix[np.ix_(target, source)] = 1
    c +=1
  return cut_matrix
def bipartitions(collection):
    collection = list(collection)
    n = len(collection)
    # Special case: no bipartitions possible if the collection has fewer than 2 elements
    if n < 2:
        return
    # Generate all combinations for the first subset of size from 1 to n-1
    for i in range(1, n):
        for first_subset in combinations(collection, i):
            second_subset = [item for item in collection if item not in first_subset]
            yield [list(first_subset), second_subset]

@system_partition_types.register("ALL_BI-PARTITIONS")
def generate_all_partitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  for partition in bipartitions(_node_indices):
    for s in gen_seqs(partition):
      cm = seq_2_cm(s, partition,_node_indices)
      yield GeneralSetPartition(
                node_indices,
                cm,
                node_labels=node_labels,
                set_partition=partition,
            )

Random K-Partition Generator

In [30]:
def stirling2(n,k):
    n1=n
    k1=k
    if n<=0:
        return 1
    elif k<=0:
        return 0
    elif (n==0 and k==0):
        return -1
    elif n!=0 and n==k:
        return 1
    elif n<k:
        return 0
    else:
        temp1=stirling2(n1-1,k1)
        temp1=k1*temp1
        return (k1*(stirling2(n1-1,k1)))+stirling2(n1-1,k1-1)
def total_partition_count(size):
  a = 0
  for i in range(1,size + 1):
    a += stirling2(size,i) * ( 3**i - 2**(i + 1) + 2)
  return a
@system_partition_types.register("RANDOM_K_PARTITIONS")
def generate_random_partitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  total = total_partition_count(len(_node_indices))
  count = 0
  sample = random.sample(range(0,total + 1),n)
  for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
    for s in gen_seqs(partition):
      if count in sample:
        cm = seq_2_cm(s, partition,_node_indices)
        yield GeneralSetPartition(
                  node_indices,
                  cm,
                  node_labels=node_labels,
                  set_partition=partition,)
      count +=1

Random Bi-Partition Generator

In [31]:
def total_bipartition_count(size):
  return stirling2(size,2)*3
@system_partition_types.register("RANDOM_BI_PARTITIONS")
def generate_random_bipartitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  total = total_bipartition_count(len(_node_indices))
  count = 0
  sample = random.sample(range(0,total + 1),n)
  for partition in bipartitions(_node_indices):
    for s in gen_seqs(partition):
      if count in sample:
        cm = seq_2_cm(s, partition,_node_indices)
        yield GeneralSetPartition(
                  node_indices,
                  cm,
                  node_labels=node_labels,
                  set_partition=partition,)
      count +=1

Network Generating Functions


In [32]:
#### NETWORK GENERATING FUNCTIONS
def normalize_rows(matrix):
    ### Scales probabilities in a matrix so that they satisfy the markov property. Proof that CI still holds not completed yet
    num_rows = matrix.shape[0]
    normalized_matrix = np.zeros_like(matrix)  # Create an empty matrix of the same shape
    for i in range(num_rows):
        row = matrix[i, :]
        current_sum = np.sum(row)
        if current_sum > 0:
            scaling_factor = 1.0 / current_sum
            normalized_row = row * scaling_factor
            normalized_matrix[i, :] = normalized_row
        else: print("Zero_Sum_Error: Problem With Values Generated")
    return normalized_matrix
def create_noisy_network(size):
    ### Obtains a random network of nodes of a given size, directly ready for computing Phi.
    return pyphi.Network(normalize_rows(pyphi.convert.state_by_node2state_by_state(np.random.rand(2**size,size))))
def create_noiseless_network(size):
  return pyphi.Network(np.random.randint(0,2,(2**size,size)))

Testing Each Partition Type:

In [36]:
size = 4
network = create_noisy_network(size)
state = [np.random.choice([0, 1]) for i in range(size)]
subsystem_cause = pyphi.Subsystem(network,state,backward_tpm=True)
subsystem_effect = pyphi.Subsystem(network,state,backward_tpm=False)

pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI"
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(f - s, "Seconds")

pyphi.config.SYSTEM_PARTITION_TYPE = "ALL_K-PARTITIONS"
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(f - s, "Seconds")

pyphi.config.SYSTEM_PARTITION_TYPE = "ALL_BI-PARTITIONS"
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(f - s, "Seconds")

pyphi.config.SYSTEM_PARTITION_TYPE = "RANDOM_K_PARTITIONS"
n = 10
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(f - s, "Seconds")

pyphi.config.SYSTEM_PARTITION_TYPE = "RANDOM_BI_PARTITIONS"
n = 10
s = time.time()
sia = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f = time.time()
print(sia)
print(f - s, "Seconds")

┌──────────────────────────────────────┐
│     SystemIrreducibilityAnalysis     │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ │
│      Subsystem:  n0,n1,n2,n3         │
│  Current state:  (0,0,0,1)           │
│            φ_s: 0.02684880305592458  │
│ Normalized φ_s: 0.008949601018641526 │
│          CAUSE:  (1,0,1,0)           │
│           II_c: 0.7977695002692182   │
│         EFFECT:  (1,0,0,0)           │
│           II_e: 0.46048472235595483  │
│   #(tied MIPs): 0                    │
│      Partition:                      │
│                 2 parts: {n0,n1n2n3} │
│                 [[0 0 0 0]           │
│                  [1 0 0 0]           │
│                  [1 0 0 0]           │
│                  [1 0 0 0]]          │
└──────────────────────────────────────┘
2.1248443126678467 Seconds
┌──────────────────────────────────────┐
│     SystemIrreducibilityAnalysis     │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ │
│      Subsystem:  n0,n1,n2,n3         │
│  Current state:  (0,0,0,1)  